# How to build a linear factor model

Algorithmic trading strategies use linear factor models to quantify the relationship between the return of an asset and the sources of risk that represent the main drivers of these returns. Each factor risk carries a premium, and the total asset return can be expected to correspond to a weighted average of these risk premia.

There are several practical applications of factor models across the portfolio management process from construction and asset selection to risk management and performance evaluation. The importance of factor models continues to grow as common risk factors are now tradeable:

- A summary of the returns of many assets by a much smaller number of factors reduces the amount of data required to estimate the covariance matrix when optimizing a portfolio
- An estimate of the exposure of an asset or a portfolio to these factors allows for the management of the resultant risk, for instance by entering suitable hedges when risk factors are themselves traded
- A factor model also permits the assessment of the incremental signal content of new alpha factors
- A factor model can also help assess whether a manager's performance relative to a benchmark is indeed due to skill in selecting assets and timing the market, or if instead, the performance can be explained by portfolio tilts towards known return drivers that can today be replicated as low-cost, passively managed funds without incurring active management fees

## Imports & Settings

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

from statsmodels.api import OLS, add_constant
import pandas_datareader.data as web

from linearmodels.asset_pricing import LinearFactorModel

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
sns.set_style('whitegrid')

## Get Data

Fama and French make updated risk factor and research portfolio data available through their [website](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html), and you can use the `pandas_datareader` package to obtain the data.

### Risk Factors

In particular, we will be using the five Fama—French factors that result from sorting stocks first into three size groups and then into two for each of the remaining three firm-specific factors. 

Hence, the factors involve three sets of value-weighted portfolios formed as 3 x 2 sorts on size and book-to-market, size and operating profitability, and size and investment. The risk factor values computed as the average returns of the portfolios (PF) as outlined in the following table:

| Label | Name                          | Description                                                                                                                                                                               |
|-------|-------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| SMB   | Small Minus Big               | Average return on the nine small stock portfolios minus the average return on the nine big stock portfolios                                                                               |
| HML   | High Minus Low                | Average return on the two value portfolios minus the average return on the two growth portfolios                                                                                          |
| RMW   | Robust minus Weak             | Average return on the two robust operating profitability portfolios minus the average return on the two weak operating profitability portfolios                                           |
| CMA   | Conservative Minus Aggressive | Average return on the two conservative investment portfolios minus the average return on the two aggressive investment portfolios                                                         |
| Rm-Rf | Excess return on the market   | Value-weight return of all firms incorporated in the US and listed on the NYSE, AMEX, or NASDAQ at the beginning of month t with 'good' data for t minus the one-month Treasury bill rate |

The Fama-French 5 factors are based on the 6 value-weight portfolios formed on size and book-to-market, the 6 value-weight portfolios formed on size and operating profitability, and the 6 value-weight portfolios formed on size and investment.

We will use returns at a monthly frequency that we obtain for the period 2010 – 2017 as follows:

In [4]:
ff_factor = 'F-F_Research_Data_5_Factors_2x3'
ff_factor_data = web.DataReader(ff_factor, 'famafrench', start='2015', end='2020-04')[0]
ff_factor_data.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 64 entries, 2015-01 to 2020-04
Freq: M
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mkt-RF  64 non-null     float64
 1   SMB     64 non-null     float64
 2   HML     64 non-null     float64
 3   RMW     64 non-null     float64
 4   CMA     64 non-null     float64
 5   RF      64 non-null     float64
dtypes: float64(6)
memory usage: 3.5 KB


In [5]:
ff_factor_data.describe()

,Mkt-RF,SMB,HML,RMW,CMA,RF
count,64.000000,64.000000,64.000000,64.000000,64.000000,64.000000
mean,0.709062,-0.326719,-0.721875,0.120469,-0.272344,0.082656
std,4.379687,2.681763,3.156461,1.417702,1.621281,0.072336
min,-13.380000,-8.380000,-13.960000,-2.890000,-3.350000,0.000000
25%,-0.362500,-2.180000,-2.212500,-0.782500,-1.447500,0.010000
50%,1.075000,0.215000,-0.900000,0.195000,-0.415000,0.075000
75%,2.857500,1.002500,0.517500,0.915000,0.402500,0.142500
max,13.650000,6.800000,8.220000,3.330000,3.780000,0.210000


### Portfolios

Fama and French also make available numerous portfolios that we can illustrate the estimation of the factor exposures, as well as the value of the risk premia available in the market for a given time period. We will use a panel of the 17 industry portfolios at a monthly frequency. 

We will subtract the risk-free rate from the returns because the factor model works with excess returns:

In [6]:
ff_portfolio = '17_Industry_Portfolios'
ff_portfolio_data = web.DataReader(ff_portfolio, 'famafrench', start='2010', end='2017-12')[0]
ff_portfolio_data = ff_portfolio_data.sub(ff_factor_data.RF, axis=0)
ff_portfolio_data.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 124 entries, 2010-01 to 2020-04
Freq: M
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Food    36 non-null     float64
 1   Mines   36 non-null     float64
 2   Oil     36 non-null     float64
 3   Clths   36 non-null     float64
 4   Durbl   36 non-null     float64
 5   Chems   36 non-null     float64
 6   Cnsum   36 non-null     float64
 7   Cnstr   36 non-null     float64
 8   Steel   36 non-null     float64
 9   FabPr   36 non-null     float64
 10  Machn   36 non-null     float64
 11  Cars    36 non-null     float64
 12  Trans   36 non-null     float64
 13  Utils   36 non-null     float64
 14  Rtail   36 non-null     float64
 15  Finan   36 non-null     float64
 16  Other   36 non-null     float64
dtypes: float64(17)
memory usage: 17.4 KB


In [7]:
ff_portfolio_data.describe()

,Food,Mines,Oil,Clths,Durbl,Chems,Cnsum,Cnstr,Steel,FabPr,Machn,Cars,Trans,Utils,Rtail,Finan,Other
count,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000
mean,0.742222,1.076667,0.171667,0.521389,0.377778,1.137500,0.565556,1.445000,0.852778,0.988889,1.286389,0.508889,1.052222,0.424167,0.937500,1.233889,0.998611
std,2.502402,8.007610,5.634105,3.541006,4.154644,5.295894,3.227620,4.356979,7.737562,3.790673,4.160907,4.053228,3.825094,3.198576,2.922586,4.507929,3.223412
min,-4.200000,-13.740000,-9.900000,-6.660000,-6.970000,-12.650000,-7.300000,-6.350000,-10.590000,-7.540000,-7.260000,-9.610000,-8.560000,-5.370000,-6.400000,-9.640000,-6.640000
25%,-0.777500,-5.577500,-3.817500,-1.865000,-1.837500,-1.630000,-1.155000,-2.105000,-4.472500,-1.585000,-1.957500,-1.177500,-1.382500,-1.967500,-0.880000,-1.355000,-0.595000
50%,0.440000,-0.860000,-1.160000,0.805000,-0.260000,1.465000,0.215000,1.605000,-0.415000,0.885000,1.645000,0.095000,0.885000,0.305000,0.760000,1.615000,0.955000
75%,2.037500,7.707500,2.837500,2.385000,3.065000,3.610000,2.275000,3.930000,3.840000,2.162500,3.657500,2.527500,3.240000,2.355000,2.770000,3.955000,2.487500
max,6.350000,15.670000,12.450000,8.720000,9.480000,16.060000,8.290000,10.650000,21.350000,9.730000,8.590000,9.780000,11.140000,7.900000,8.720000,12.790000,9.080000


### Equity Data

In [8]:
with pd.HDFStore('../data/crypto.h5') as store:
    print(store.info())
    prices = store['/crypto/caggle/prices'].close
    market = store['/coingecko/top100/market'].drop_duplicates()
    cats = store['/coingecko/top100/cats'].drop_duplicates()

prices.index.rename(['date', 'ticker', 'base'], inplace=True)
prices = prices.droplevel('base')
prices = prices.unstack('ticker')
prices.tail(3)

<class 'pandas.io.pytables.HDFStore'>
File path: ../data/crypto.h5
/coingecko/top100/cats                                   frame        (shape->[100,49])                                                                    
/coingecko/top100/market                                 frame        (shape->[100,12])                                                                    
/crypto/caggle/prices                                    frame        (shape->[48918134,5])                                                                
/engineered_features                                     frame_table  (typ->appendable_multi,nrows->948,ncols->35,indexers->[index],dc->[date,base,symbol])
/engineered_features/meta/values_block_2/meta            series_table (typ->appendable,nrows->5,ncols->1,indexers->[index],dc->[values])                   


ticker,aave,ada,algo,atom,avax,bch,bsv,btc,btt,dai,...,trx,uni,usdc,usdt,vet,wbtc,xlm,xmr,xrp,xtz
date,,,,,,,,,,,,,,,,,,,,,
2021-05-18 07:02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-18 07:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-18 07:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
market.index.rename(['ticker', 'base'], inplace=True)
market = market.droplevel('base')
market

,id,market_cap,name,genesis_date,market_cap_rank,hashing_algorithm,coingecko_rank,coingecko_score,developer_score,community_score,liquidity_score,public_interest_score
ticker,,,,,,,,,,,,
btc,bitcoin,851165633926,Bitcoin,2009-01-03,1,SHA-256,2,81.149,98.874,74.606,100.084,0.0
eth,ethereum,408663307732,Ethereum,2015-07-30,2,Ethash,3,78.085,97.194,65.231,99.843,0.0
bnb,binancecoin,81684939616,Binance Coin,2017-07-08,3,None,5,67.430,73.243,66.596,80.872,0.0
xrp,ripple,77810326837,XRP,NaT,4,None,9,65.219,71.122,54.329,86.490,0.0
ada,cardano,67995769831,Cardano,NaT,5,None,6,66.641,70.441,61.873,86.307,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
icx,icon,1263863907,ICON,2017-09-19,96,None,66,50.558,57.689,43.226,56.885,0.0
arrr,pirate-chain,1234724958,Pirate Chain,2018-08-29,97,Equihash,133,43.566,55.799,36.531,31.661,0.0
pax,paxos-standard,1232338318,Paxos Standard,NaT,98,None,168,40.076,47.966,8.260,55.094,0.0


In [10]:
cats.index.rename(['ticker', 'base'], inplace=True)
cats = cats.droplevel('base')
cats

,Analytics,Artificial Intelligence,Asset-backed Tokens,Automated Market Maker (AMM),Avalanche Ecosystem,Binance Smart Chain Ecosystem,Business Platform,Business Services,Centralized Exchange Token (CEX),Communication,...,Stablecoins,Storage,Synthetic Issuer,Terra Ecosystem,Tokenized BTC,USD Stablecoin,Wrapped-Tokens,Yearn Ecosystem,Yield Aggregator,Yield Farming
ticker,,,,,,,,,,,,,,,,,,,,,
btc,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
eth,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bnb,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
doge,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
usdt,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
crv,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
hnt,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
prices = prices.resample('M').last()
prices.tail()


ticker,aave,ada,algo,atom,avax,bch,bsv,btc,btt,dai,...,trx,uni,usdc,usdt,vet,wbtc,xlm,xmr,xrp,xtz
date,,,,,,,,,,,,,,,,,,,,,
2021-01-31,NaN,0.34585,0.65084,8.1545,NaN,NaN,173.99,33127.00000,0.000380,1.0002,...,0.031628,17.708,1.00000,1.00090,0.025476,31727.0,0.30710,137.81,0.49637,2.8433
2021-02-28,NaN,1.30990,1.01790,17.6740,NaN,NaN,177.42,45215.00000,0.001160,1.0038,...,0.045679,22.239,1.00060,1.00280,0.040396,45171.0,0.40575,219.31,0.41567,3.4248
2021-03-31,NaN,1.19220,1.37100,19.0470,NaN,NaN,217.54,58732.00000,0.005049,0.9999,...,0.092800,28.028,1.00010,1.00010,0.087330,58363.0,0.40412,246.91,0.56993,4.8276
2021-04-30,NaN,1.35160,1.40100,22.6880,NaN,NaN,323.21,57649.15327,0.007235,0.9991,...,0.132130,40.651,0.99840,0.99953,0.200700,57426.0,0.52884,422.17,1.59090,5.6034
2021-05-31,647.82,2.11680,1.34530,22.7640,36.718,1115.5,293.13,45745.00000,0.005635,0.9994,...,0.117770,36.000,0.99921,0.99949,0.170480,45016.0,0.67117,356.10,1.54150,5.5380


In [12]:
prices=prices.stack(['ticker'])
shared = prices.index.get_level_values('ticker').intersection(market.index.get_level_values('ticker'))\
    .intersection(cats.index.get_level_values('ticker'))
len(shared), shared

(22,
 Index(['btc', 'eth', 'xmr', 'miota', 'dai', 'mkr', 'xlm', 'vet', 'bsv', 'usdt',
        'btt', 'atom', 'wbtc', 'okb', 'doge', 'dot', 'uni', 'fil', 'sol',
        'aave', 'link', 'luna'],
       dtype='object', name='ticker'))

In [13]:
cryptos = market.loc[shared, :]
cryptos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22 entries, btc to luna
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     22 non-null     object        
 1   market_cap             22 non-null     int64         
 2   name                   22 non-null     object        
 3   genesis_date           7 non-null      datetime64[ns]
 4   market_cap_rank        22 non-null     int64         
 5   hashing_algorithm      22 non-null     object        
 6   coingecko_rank         22 non-null     int64         
 7   coingecko_score        22 non-null     float64       
 8   developer_score        22 non-null     float64       
 9   community_score        22 non-null     float64       
 10  liquidity_score        22 non-null     float64       
 11  public_interest_score  22 non-null     float64       
dtypes: datetime64[ns](1), float64(5), int64(3), object(3)
memory usage:

In [14]:
categories = cats.loc[shared, :]
categories.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22 entries, btc to luna
Data columns (total 49 columns):
 #   Column                              Non-Null Count  Dtype
---  ------                              --------------  -----
 0   Analytics                           22 non-null     int64
 1   Artificial Intelligence             22 non-null     int64
 2   Asset-backed Tokens                 22 non-null     int64
 3   Automated Market Maker (AMM)        22 non-null     int64
 4   Avalanche Ecosystem                 22 non-null     int64
 5   Binance Smart Chain Ecosystem       22 non-null     int64
 6   Business Platform                   22 non-null     int64
 7   Business Services                   22 non-null     int64
 8   Centralized Exchange Token (CEX)    22 non-null     int64
 9   Communication                       22 non-null     int64
 10  Compound Tokens                     22 non-null     int64
 11  Cosmos Ecosystem                    22 non-null     int64
 12  Cryptocurre

In [18]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.IndexSlice.html#prices.index.get_level_values('symbol').isin(shared)
prices = prices.loc[(slice(None), shared)]
prices.info()


AttributeError: 'Series' object has no attribute 'info'

In [ ]:
sectors = cats.filter(prices.columns, axis=0).to_dict()
sectors

In [ ]:
prices = prices.filter(sectors.keys()).dropna(how='all', axis=1)
prices.info()

In [ ]:
returns = prices.pct_change().mul(100).to_period('M')
returns = returns.dropna(how='all').dropna(axis=1)
returns.info()

### Align data

In [ ]:
ff_factor_data = ff_factor_data.loc[returns.index]
ff_portfolio_data = ff_portfolio_data.loc[returns.index]

In [ ]:
ff_factor_data.describe()

### Compute excess Returns

In [ ]:
excess_returns = returns.sub(ff_factor_data.RF, axis=0)
excess_returns.info()

In [ ]:
excess_returns = excess_returns.clip(lower=np.percentile(excess_returns, 1),
                                     upper=np.percentile(excess_returns, 99))

## Fama-Macbeth Regression

Given data on risk factors and portfolio returns, it is useful to estimate the portfolio's exposure, that is, how much the risk factors drive portfolio returns, as well as how much the exposure to a given factor is worth, that is, the what market's risk factor premium is. The risk premium then permits to estimate the return for any portfolio provided the factor exposure is known or can be assumed.

In [ ]:
ff_portfolio_data.info()

In [ ]:
ff_factor_data = ff_factor_data.drop('RF', axis=1)
ff_factor_data.info()

To address the inference problem caused by the correlation of the residuals, Fama and MacBeth proposed a two-step methodology for a cross-sectional regression of returns on factors. The two-stage Fama—Macbeth regression is designed to estimate the premium rewarded for the exposure to a particular risk factor by the market. The two stages consist of:

- First stage: N time-series regression, one for each asset or portfolio, of its excess returns on the factors to estimate the factor loadings.

- Second stage: T cross-sectional regression, one for each time period, to estimate the risk premium.

See corresponding section in Chapter 7 of [Machine Learning for Trading](https://www.amazon.com/Hands-Machine-Learning-Algorithmic-Trading-ebook/dp/B07JLFH7C5/ref=sr_1_2?ie=UTF8&qid=1548455634&sr=8-2&keywords=machine+learning+algorithmic+trading) for details.

Now we can compute the factor risk premia as the time average and get t-statistic to assess their individual significance, using the assumption that the risk premia estimates are independent over time.

If we had a very large and representative data sample on traded risk factors we could use the sample mean as a risk premium estimate. However, we typically do not have a sufficiently long history to and the margin of error around the sample mean could be quite large. 

The Fama—Macbeth methodology leverages the covariance of the factors with other assets to determine the factor premia. The second moment of asset returns is easier to estimate than the first moment, and obtaining more granular data improves estimation considerably, which is not true of mean estimation.

### Step 1: Factor Exposures

We can implement the first stage to obtain the 17 factor loading estimates as follows:

In [ ]:
betas = []
for industry in ff_portfolio_data:
    step1 = OLS(endog=ff_portfolio_data.loc[ff_factor_data.index, industry], 
                exog=add_constant(ff_factor_data)).fit()
    betas.append(step1.params.drop('const'))

In [ ]:
betas = pd.DataFrame(betas, 
                     columns=ff_factor_data.columns, 
                     index=ff_portfolio_data.columns)
betas.info()

### Step 2: Risk Premia

For the second stage, we run 96 regressions of the period returns for the cross section of portfolios on the factor loadings

In [ ]:
lambdas = []
for period in ff_portfolio_data.index:
    step2 = OLS(endog=ff_portfolio_data.loc[period, betas.index], 
                exog=betas).fit()
    lambdas.append(step2.params)

In [ ]:
lambdas = pd.DataFrame(lambdas, 
                       index=ff_portfolio_data.index,
                       columns=betas.columns.tolist())
lambdas.info()

In [ ]:
lambdas.mean().sort_values().plot.barh(figsize=(12, 4))
sns.despine()
plt.tight_layout();

In [ ]:
t = lambdas.mean().div(lambdas.std())
t

#### Results

In [ ]:
window = 24  # months
ax1 = plt.subplot2grid((1, 3), (0, 0))
ax2 = plt.subplot2grid((1, 3), (0, 1), colspan=2)
lambdas.mean().sort_values().plot.barh(ax=ax1)
lambdas.rolling(window).mean().dropna().plot(lw=1,
                                             figsize=(14, 5),
                                             sharey=True,
                                             ax=ax2)
sns.despine()
plt.tight_layout()

In [ ]:
window = 24  # months
lambdas.rolling(window).mean().dropna().plot(lw=2,
                                             figsize=(14, 7),
                                             subplots=True,
                                             sharey=True)
sns.despine()
plt.tight_layout()

## Fama-Macbeth with the LinearModels library

The linear_models library extends statsmodels with various models for panel data and also implements the two-stage Fama—MacBeth procedure:

In [ ]:
mod = LinearFactorModel(portfolios=ff_portfolio_data, 
                        factors=ff_factor_data)
res = mod.fit()
print(res)

In [ ]:
print(res.full_summary)

This provides us with the same result:

In [ ]:
lambdas.mean()